In [3]:
!curl -L https://www.dropbox.com/s/rndzndlgpw3coiq/kaggle-download.zip -o kaggle-download.zip
!unzip kaggle-download.zip -d kaggle_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17    0    17    0     0     43      0 --:--:-- --:--:-- --:--:--    43
100   340  100   340    0     0    437      0 --:--:-- --:--:-- --:--:--   437
100   534    0   534    0     0    391      0 --:--:--  0:00:01 --:--:--  1158
100  515M  100  515M    0     0  56.5M      0  0:00:09  0:00:09 --:--:-- 71.6M
Archive:  kaggle-download.zip
  inflating: kaggle_data/metaData_taxistandsID_name_GPSlocation.csv  
  inflating: kaggle_data/sampleSubmission.csv  
  inflating: kaggle_data/test_public.csv  
  inflating: kaggle_data/train.csv   


In [1]:
# DO ALL IMPORTS HERE

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = pd.read_csv('kaggle_data/train.csv')

# Calculate Z-scores 
#z_scores = stats.zscore(train_data)
# Define a threshold to identify outliers
threshold = 3
# Remove outliers
#train_data = train_data[(z_scores < threshold).all(axis=1)]

train_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [5]:
test_data = pd.read_csv('kaggle_data/test_public.csv')
test_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [6]:
train_data['TRAVEL_TIME'] = train_data['POLYLINE'].apply(lambda x: max(len(x.split('[')) - 2, 0) * 15)
train_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,TRAVEL_TIME
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",345
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",285
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",975
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",645
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",435


In [7]:
from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
train_data[["YR", "MON", "DAY", "HR", "WK"]] = train_data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [ ]:
y = train_data["TRAVEL_TIME"]
train_data = train_data.drop(columns=["TRIP_ID", "TAXI_ID", "POLYLINE", "TRAVEL_TIME", "MISSING_DATA"])
train_data["ORIGIN_CALL"] = train_data["ORIGIN_CALL"].fillna(-1)
train_data["ORIGIN_STAND"] = train_data["ORIGIN_STAND"].fillna(-1)



# This will remove rows with any NaN value, which were outliers
#train_oh = train_oh.dropna()


In [11]:
# One Hot encode:
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_data = encoder.fit_transform(train_data[['CALL_TYPE']])
column_names = encoder.get_feature_names_out(['CALL_TYPE'])
train_oh = pd.concat([train_data, pd.DataFrame(encoded_data, columns=column_names)], axis=1)
train_oh = train_oh.drop(columns=['CALL_TYPE', 'DAY_TYPE', "CALL_TYPE_B", "CALL_TYPE_C"])

scaler = StandardScaler()

train_oh = scaler.fit_transform(train_oh)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
train_data

,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TIMESTAMP,DAY_TYPE,YR,MON,DAY,HR,WK
0,C,-1.0,-1.0,1372636858,A,2013,7,1,0,0
1,B,-1.0,7.0,1372637303,A,2013,7,1,0,0
2,C,-1.0,-1.0,1372636951,A,2013,7,1,0,0
3,C,-1.0,-1.0,1372636854,A,2013,7,1,0,0
4,C,-1.0,-1.0,1372637091,A,2013,7,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1710665,C,-1.0,-1.0,1404171463,A,2014,6,30,23,0
1710666,C,-1.0,-1.0,1404171367,A,2014,6,30,23,0
1710667,C,-1.0,-1.0,1388745716,A,2014,1,3,10,4
1710668,B,-1.0,12.0,1404141826,A,2014,6,30,15,0


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_oh, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(np.array(y_val), dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create a DataLoader
batch_size = 256
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegression, self).__init__()
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
model = MLP(input_dim=train_oh.shape[1]).to(device)

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
torch.cuda.empty_cache()

In [ ]:
train_losses = []
val_losses = []

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for batch, (X_batch, y_batch) in enumerate(train_dataloader):
        optimizer.zero_grad()

        y_pred = model(X_batch)
        y_batch = y_batch.view(-1, 1)
        loss = loss_function(y_pred, y_batch)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

    # switch model to evaluation mode to prevent updating of weights
    model.eval()
    with torch.no_grad():
        running_val_loss = 0
        for X_val_batch, y_val_batch in val_dataloader:
            y_val_pred = model(X_val_batch)
            y_val_batch = y_val_batch.view(-1, 1)
            val_loss = loss_function(y_val_pred, y_val_batch)
            running_val_loss += val_loss.item()

    train_loss = np.sqrt(running_loss / len(train_dataloader))
    val_loss = np.sqrt(running_val_loss / len(val_dataloader))
    print(f'Epoch: {epoch}, Training RMSE: {train_loss:.3f}, Validation RMSE: {val_loss:.3f}')

    # Append losses to lists
    train_losses.append(train_loss)
    val_losses.append(val_loss)

Epoch: 0, Training RMSE: 695.722, Validation RMSE: 680.337
Epoch: 1, Training RMSE: 677.463, Validation RMSE: 679.645
Epoch: 2, Training RMSE: 677.001, Validation RMSE: 679.311
Epoch: 3, Training RMSE: 676.745, Validation RMSE: 679.491
Epoch: 4, Training RMSE: 676.512, Validation RMSE: 678.825
Epoch: 5, Training RMSE: 676.318, Validation RMSE: 678.992
Epoch: 6, Training RMSE: 676.163, Validation RMSE: 678.349
Epoch: 7, Training RMSE: 675.982, Validation RMSE: 678.319
Epoch: 8, Training RMSE: 675.839, Validation RMSE: 678.072
Epoch: 9, Training RMSE: 675.684, Validation RMSE: 678.033


In [ ]:
losses = []
model.eval()
with torch.no_grad():
    running_val_loss = 0
    for x, y in zip(X_train, y_train):
        y_pred = model(x)
        y = y.view(-1)
        loss = loss_function(y_pred, y)
        losses.append(loss.item())


In [ ]:
sorted_losses = list(enumerate(losses))
sorted_losses.sort(key=lambda x: x[1])

In [ ]:
top_ten = sorted_losses[-10:]
trip_ids = []
for loss in top_ten:
    trip_ids.append(loss[0])

In [ ]:
train_data['TRIP_ID'].iloc[trip_ids].to_csv('highest_training_IDs.csv', index=False)

In [ ]:
# Plot training and validation losses
plt.figure(figsize=(10,5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (RMSE)')
plt.legend()
plt.title('Training and Validation Losses Over Epochs')
plt.show()

In [ ]:
# Preprocess the test data
original_test_data = test_data.copy()
test_data[["YR", "MON", "DAY", "HR", "WK"]] = test_data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
test_data = test_data.drop(columns=["TRIP_ID", "TAXI_ID", "MISSING_DATA"])
test_data["ORIGIN_CALL"] = test_data["ORIGIN_CALL"].fillna(-1)
test_data["ORIGIN_STAND"] = test_data["ORIGIN_STAND"].fillna(-1)

# One-hot encode
encoded_data_test = encoder.transform(test_data[['CALL_TYPE']])
test_data_oh = pd.concat([test_data, pd.DataFrame(encoded_data_test, columns=column_names)], axis=1)
test_data_oh = test_data_oh.drop(columns=['CALL_TYPE', 'DAY_TYPE', "CALL_TYPE_B", "CALL_TYPE_C"])

# Scale the data
test_data_oh = scaler.transform(test_data_oh)

# Convert to tensor
X_test = torch.tensor(test_data_oh, dtype=torch.float32).to(device)

# Predict
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test)
y_pred_test = y_pred_test.view(-1)

# Convert to numpy array
y_pred_test_np = y_pred_test.cpu().numpy()

# Create a DataFrame with the trip IDs and predicted travel times
submission = pd.DataFrame({'TRIP_ID': original_test_data['TRIP_ID'], 'TRAVEL_TIME': y_pred_test_np})

# Save to CSV
submission.to_csv('submission.csv', index=False)


In [ ]:
test_data